In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/19 20:18:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

In [2]:
val regex = ".*evidence_(.*)_validation_.*".r
val steps = RESULTS_DIR.resolve("errors").toFile.listFiles.map(_.toString).map({
    case regex(k) => k
    case _ => ""
}).toSet

regex: scala.util.matching.Regex = .*evidence_(.*)_validation_.*
steps: Set[String] = Set("disease_id", "schema", "target_id", "data_source")

In [3]:
def loadSummary(step: String) = ss.read
    .parquet(RESULTS_DIR.resolve(s"errors/evidence_${step}_validation_summary.parquet").toString)
    .withColumn("step", lit(step))

defined function loadSummary

In [4]:
val df = steps.map(loadSummary).reduce(_.union(_))

parquet at cmd2.sc:2

1 / 1

parquet at cmd2.sc:2

1 / 1

parquet at cmd2.sc:2

1 / 1

parquet at cmd2.sc:2

1 / 1

df: org.apache.spark.sql.package.DataFrame = [sourceID: string, reason: string ... 2 more fields]

In [5]:
df.show(1000, false)

show at cmd4.sc:1

1 / 1

show at cmd4.sc:1

4 / 4

show at cmd4.sc:1

20 / 20

show at cmd4.sc:1

31 / 31

+------------------+------------------------------+-------+-----------+
|sourceID          |reason                        |count  |step       |
+------------------+------------------------------+-------+-----------+
|uniprot_literature|id_not_found                  |10     |disease_id |
|gene2phenotype    |id_not_found                  |3      |disease_id |
|phewas_catalog    |id_not_found                  |8941   |disease_id |
|cancer_gene_census|null                          |96274  |disease_id |
|genomics_england  |id_not_found                  |522    |disease_id |
|eva_somatic       |id_not_found                  |10     |disease_id |
|phenodigm         |id_not_found                  |714    |disease_id |
|europepmc         |id_not_found                  |106933 |disease_id |
|cancer_gene_census|id_not_found                  |10     |disease_id |
|crispr            |null                          |1844   |disease_id |
|reactome          |id_not_found                  |30     |disea

### Record Invalidation Cause Frequency by Source

In [6]:
df.filter($"reason".isNotNull).show(100, false)

show at cmd5.sc:1

1 / 1

show at cmd5.sc:1

4 / 4

show at cmd5.sc:1

20 / 20

show at cmd5.sc:1

31 / 31

+------------------+------------------------------+------+----------+
|sourceID          |reason                        |count |step      |
+------------------+------------------------------+------+----------+
|uniprot_literature|id_not_found                  |10    |disease_id|
|gene2phenotype    |id_not_found                  |3     |disease_id|
|phewas_catalog    |id_not_found                  |8941  |disease_id|
|genomics_england  |id_not_found                  |522   |disease_id|
|eva_somatic       |id_not_found                  |10    |disease_id|
|phenodigm         |id_not_found                  |714   |disease_id|
|europepmc         |id_not_found                  |106933|disease_id|
|cancer_gene_census|id_not_found                  |10    |disease_id|
|reactome          |id_not_found                  |30    |disease_id|
|uniprot           |id_not_found                  |58    |disease_id|
|progeny           |id_not_found                  |87    |disease_id|
|gwas_catalog      |